# MeshCat Python

In [1]:
import numpy as np
import os
import time

# CHANGE THIS DIRECTOR TO AVOID REBUILDING
my_dir = '/Users/ethanweber/Documents/RobotLocomotionGroup/meshcat-python'
os.chdir(my_dir+'/src')

import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

In [2]:
# Create a new visualizer
vis = meshcat.Visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


By default, creating the `Visualizer` will start up a meshcat server for you in the background. The easiest way to open the visualizer is with its ``open`` method:

In [3]:
# vis.open()

If ``vis.open()`` does not work for you, you can also point your browser to the server's URL:

In [4]:
vis.url()

'http://127.0.0.1:7000/static/'

In [5]:
vis.jupyter_cell()

## Cart-Pole Simluation

In [227]:
cart = vis["cart"]
pivot = cart["pivot"]
pole = pivot["pole"]

In [283]:
# specify starting config parameters
cart_dim = [0.5, 0.3, 0.2]
pol_dim = [1.0, 0.05, 0.05]
theta = np.pi -.3 # vertical
x_pos = 0
x_vel = 0
x_acc = 0
theta_vel = 0
theta_acc = 0

In [284]:
# draw initial positions
cart.set_object(g.Box(cart_dim))
pole.set_object(g.Box(pol_dim))
# attach pole to body
pole.set_transform(tf.translation_matrix([cart_dim[0], 0, 0]))
# set the angle of the pole
pivot.set_transform(tf.rotation_matrix(-theta + np.pi/2.0, [0, 1, 0]))
# draw the cart x position
cart.set_transform(tf.translation_matrix([x_pos, 0, 0]))

In [285]:
# specify parameters of the cart
mp = .5 # 10 kg for pole mass
mc = .2 # mass of the pole
gr = 9.81 # m / s^2 gravity
fric_b = 0.1 # friction constant b (added to equations)
l = pol_dim[0] # in meters for length
f = 0 # force applied on ground

In [286]:
# set the PID control
setpoint = np.pi
error = 0 # just the error
error_dot = 0 # change in error
error_int = 0 # for integration

kp = 100.0
ki = 0.0
kd = 1000.0


In [287]:
def get_x_acc(f, x_vel, mp, mc, gr, theta, theta_vel, l):
    first = (1.0/(mc+mp*(np.sin(theta)**2)))
    second = ((f+fric_b*x_vel)+mp*np.sin(theta)*(l*theta_vel**2+gr*np.cos(theta)))
    return first*second
def get_theta_acc(f, x_vel, mp, mc, gr, theta, theta_vel, l):
    first = (1.0/(l*(mc+mp*(np.sin(theta)**2))))
    second = -(f+fric_b*x_vel)*np.cos(theta)-mp*l*theta_vel**2*np.cos(theta)*np.sin(theta)-(mc+mp)*gr*np.sin(theta)
    return first*second

In [288]:
sec = 10 # seconds
steps_per_sec = 1000 # discrete steps in a second
num_steps = sec*steps_per_sec # total number of steps
sec_per_step = sec/num_steps # number of seconds per step
for t in np.linspace(0,sec,num_steps):
    
    # set the error values
    current_error = setpoint - theta
    error_dot = current_error - error
    error_int += current_error
    error = current_error
    
    u = kp*(error) + kd*error_dot + ki*error_int
    
    
    # set the force using the control law
#     u = (mp+mc)*x_acc-fric_b*x_vel-mp*l*theta_acc
    f = u
    
    # update x values
    x_acc = get_x_acc(f, x_vel, mp, mc, gr, theta, theta_vel, l)
    x_vel += x_acc*sec_per_step
    x_pos += x_vel*sec_per_step
    
    # update theta values
    theta_acc = get_theta_acc(f, x_vel, mp, mc, gr, theta, theta_vel, l)
    theta_vel += theta_acc*sec_per_step
    theta += theta_vel*sec_per_step
    theta = theta%(2*np.pi)
    
    cart.set_transform(tf.translation_matrix([x_pos, 0, 0]))
    pivot.set_transform(tf.rotation_matrix(-theta + np.pi/2.0, [0, 1, 0]))
    time.sleep(sec_per_step)